In [81]:
import pandas as pd
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
import chardet    
rawdata = open('source.csv', 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)

Big5


In [83]:
df = pd.read_csv('source.csv',encoding=r'Big5')

In [84]:
df['發生時段']

0       08~10
1       00~02
2       00~02
3       06~08
4       10~12
        ...  
3267    09~11
3268    12~14
3269    06~08
3270    15~17
3271    18~20
Name: 發生時段, Length: 3272, dtype: object

In [85]:
time_zone = []
for r in df['發生時段']:
    if(r == '00~02' or r == '03~05' or r == '04~06' or r == '05~07'):
        time_zone.append('midnight')
    elif(r == '06~08' or r =='08~10' or r == '09~11' or r == '10~12' or r == '11~13'):
        time_zone.append('morning')
    elif(r == '12~14' or r == '14~16' or r == '15~17' or r == '15~18' or r == '16~18' or r == '17~19'):
        time_zone.append('afternoon')
    else:
        time_zone.append('night')


In [86]:
df['time_zone'] = time_zone

In [87]:
df.loc[df['time_zone']=='morning'].to_csv('morning.csv',encoding='utf-8-sig')

In [88]:
list = []

In [89]:
for r in df['發生地點']:
    list.append(r[0:6])
df['locate'] = list

In [90]:
train, test = train_test_split(df, test_size = 0.2)

In [91]:
BOW_vectorizer = CountVectorizer() 

In [92]:
BOW_vectorizer.fit(train['locate'])
train_data_BOW_features = BOW_vectorizer.transform(train['locate'])
test_data_BOW_features = BOW_vectorizer.transform(test['locate'])

In [93]:
train_data_BOW_features

<2617x12 sparse matrix of type '<class 'numpy.int64'>'
	with 2617 stored elements in Compressed Sparse Row format>

In [94]:
feature_names = BOW_vectorizer.get_feature_names()
feature_names

['臺北市中山區',
 '臺北市中正區',
 '臺北市信義區',
 '臺北市內湖區',
 '臺北市北投區',
 '臺北市南港區',
 '臺北市士林區',
 '臺北市大同區',
 '臺北市大安區',
 '臺北市文山區',
 '臺北市松山區',
 '臺北市萬華區']

In [95]:
import nltk
import jieba
# build analyzers (bag-of-words)
BOW = CountVectorizer(tokenizer=jieba.cut) 

# apply analyzer to training data
BOW.fit(train['發生地點'])

train_data_BOW_features = BOW.transform(train['time_zone'])

## check dimension
train_data_BOW_features.shape

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(2617, 1402)

In [96]:
feature_names = BOW.get_feature_names()
feature_names

[' ',
 '(',
 ')',
 '002',
 '003',
 '004',
 '006',
 '007',
 '009',
 '01',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017',
 '018',
 '019',
 '020',
 '022',
 '025',
 '026',
 '027',
 '028',
 '029',
 '03',
 '030',
 '031',
 '032',
 '1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '144',
 '145',
 '146',
 '147',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '19

In [97]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW.transform(train['locate'])
y_train = train['time_zone']

X_test = BOW.transform(test['locate'])
y_test = test['time_zone']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (2617, 1402)
y_train.shape:  (2617,)
X_test.shape:  (655, 1402)
y_test.shape:  (655,)


In [98]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:20]

array(['night', 'night', 'night', 'night', 'night', 'night', 'afternoon',
       'night', 'afternoon', 'morning', 'night', 'night', 'night',
       'afternoon', 'night', 'night', 'night', 'night', 'afternoon',
       'night'], dtype=object)

In [99]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

training accuracy: 0.34
testing accuracy: 0.32


In [100]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['afternoon' 'midnight' 'morning' 'night']

## Before convert
y_train[0:4]:
 1401    afternoon
874         night
1504    afternoon
76        morning
Name: time_zone, dtype: object

y_train.shape:  (2617,)
y_test.shape:  (655,)


## After convert
y_train[0:4]:
 [[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]

y_train.shape:  (2617, 4)
y_test.shape:  (655, 4)


In [101]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  1402
output_shape:  4


In [102]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1402)]            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                89792     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 260       
_________________________________________________________________
softmax_1 (Softmax)          (None, 4)                 0   

In [103]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data = (X_test, y_test))
print('training finish')

Epoch 1/25


C:\Users\ASUS\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/dense_3/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/dense_3/embedding_lookup_sparse/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/dense_3/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


82/82 [==============================] - 0s 2ms/step - loss: 1.3464 - accuracy: 0.3160 - val_loss: 1.3683 - val_accuracy: 0.2916
Epoch 2/25
82/82 [==============================] - 0s 1ms/step - loss: 1.3375 - accuracy: 0.3225 - val_loss: 1.3600 - val_accuracy: 0.3252
Epoch 3/25
82/82 [==============================] - 0s 2ms/step - loss: 1.3339 - accuracy: 0.3256 - val_loss: 1.3664 - val_accuracy: 0.3359
Epoch 4/25
82/82 [==============================] - 0s 956us/step - loss: 1.3325 - accuracy: 0.3217 - val_loss: 1.3608 - val_accuracy: 0.3252
Epoch 5/25
82/82 [==============================] - 0s 1ms/step - loss: 1.3315 - accuracy: 0.3317 - val_loss: 1.3613 - val_accuracy: 0.3252
Epoch 6/25
82/82 [==============================] - 0s 1ms/step - loss: 1.3302 - accuracy: 0.3298 - val_loss: 1.3629 - val_accuracy: 0.3221
Epoch 7/25
82/82 [==============================] - 0s 3ms/step - loss: 1.3317 - accuracy: 0.3301 - val_loss: 1.3642 - val_accuracy: 0.3176
Epoch 8/25
82/82 [===========

In [104]:
## predict
pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]

array([[0.22167681, 0.18954672, 0.25963175, 0.32914472],
       [0.25778177, 0.14612627, 0.2759419 , 0.32015008],
       [0.30781367, 0.12419466, 0.23310699, 0.3348847 ],
       [0.24391991, 0.20139644, 0.24855572, 0.30612788],
       [0.3080414 , 0.12868972, 0.24701944, 0.3162494 ]], dtype=float32)

In [105]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['night', 'night', 'night', 'night', 'night'], dtype=object)

In [106]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

testing accuracy: 0.32


---如何預測---

In [107]:
test = BOW.transform(['乾你媽的'])



In [108]:
DT_model.predict(test)[0]

'night'

In [118]:
target = df['發生地點']

In [119]:
target_bow = BOW.transform(target)

In [120]:
predict = DT_model.predict(target_bow)

In [128]:
predict = pd.Series(predict)

In [135]:
res = pd.concat([target,predict], axis=1, ignore_index=True)

In [139]:
res.to_csv('result.csv', index=False, encoding='utf-8-sig')